<a href="https://colab.research.google.com/github/davidfague/Stylized-ReducedOrder-L5-Model/blob/main/Input_Modulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Functions used for creating inputs

build_network.py
https://github.com/latimerb/L5NeuronSimulation/blob/dea91c052102bb4c2a58b860ebed88b3af4a24bc/L5NeuronSimulation/FullSimulation/build_network.py#L626

In [ ]:
        def _make_rasters(self):
                """Generates excitatory and inhibitory input rasters
                """    
                np.random.seed(self.seed + 9)
                self._gen_exc_spikes('exc_stim_spikes.h5')

                inh_frs = self.params["inh_frs"]

                #Makes perisomatic inhibitory raster.
                np.random.seed(self.seed + 10)
                self._gen_inh_spikes(self.n_soma_inh + self.n_prox_dend_inh, 
                                     inh_frs["proximal"]["m"], 
                                     inh_frs["proximal"]["s"], 
                                     inh_frs["proximal"]["rhythmicity"],
                                     "prox_inh_stim", 
                                     'prox_inh_stim_spikes.h5')
                
                #Makes dendritic inhibitory raster.
                np.random.seed(self.seed + 11)
                self._gen_inh_spikes(self.n_apic_inh + self.n_dend_inh, 
                                     inh_frs["distal"]["m"], 
                                     inh_frs["distal"]["s"],
                                     inh_frs["distal"]["rhythmicity"],
                                     "dist_inh_stim", 
                                     'dist_inh_stim_spikes.h5')


        #Generates the spike raster for a given group.
        #The group has the same noise.
        def _gen_group_spikes(writer, group, seconds, start_time, dist):
                """Generates and writes to a h5 file the given functional group's spike trains
                Parameters
                ----------
                writer : SonataWriter
                    how the spike trains are saved
                group : FunctionalGroup
                    the functional group that the spike trains are being made for
                seconds : float
                    length of the spike trains in seconds
                start_time : float
                    what time (ms) the spike trains should start at
                dist : func
                    function for random distribution used for an individual cell's firing rate
                """                
                z = make_noise(num_samples=(int(seconds*1000))-1,num_traces=1)#generates the noise trace common to each cell in the functional group.
                make_save_spikes(writer, True, dist(size=group.n_cells), numUnits=group.n_cells,
                        rateProf=np.tile(z[0,:],(group.n_cells,1)),start_id=group.start_id,
                        start_time=start_time)

        #Creates the excitatory input raster from the functional groups.
        def _gen_exc_spikes(self, fname):
                """Generates the excitatory input raster for all of the functional groups
                Parameters
                ----------
                fname : str
                    name of the file to save the rasters in (.h5)
                """    
                #distribution used for generating excitatory firing rates.    
                levy_dist = partial(st.levy_stable.rvs, alpha=1.37, beta=-1.00, loc=0.92, scale=0.44, size=1)

                length = self.params["time"]["stop"] - self.params["time"]["start"]
                buffer = self.params["time"]["start"]

                writer = SonataWriter(fname, ["spikes", "exc_stim"], ["timestamps", "node_ids"], [np.float, np.int])

                for group in (self.dend_groups + self.apic_groups):
                        SimulationBuilder._gen_group_spikes(writer, group, length, buffer*1000, levy_dist)
                

        #Blocks off the bottom of a normal distribution.
        def _norm_rvs(mean, std):
                """Generates a random float from a normal distribution with a near zero minimum
                Parameters
                ----------
                mean : float
                    mean of the distribution
                std : float
                    standard deviation of the distribution
                Returns
                -------
                float
                    random float
                """                
                return max(st.norm.rvs(loc=mean, scale=std, size=1), 0.001)

        # #Makes a spike raster with each cell having its own noise trace.
        # def gen_inh_spikes(n_cells, mean_fr, std_fr, key, file, times):
        #         # node_ids = []
        #         # timestamps = []

        #         length = times[1] - times[0]
        #         buffer = times[0]

        #         writer = SonataWriter(file, ["spikes", key], ["timestamps", "node_ids"], [np.float, np.int])

        #         z = make_noise(num_samples=(int(length*1000))-1,num_traces=1)
        #         make_save_spikes(writer, False, partial(positive_normal, mean=mean_fr, std=std_fr), numUnits=n_cells,rateProf=z[0,:],start_time=buffer*1000)

        #Creates a spike raster with each cell having the same noise coming from the a shifted average of excitation.
        def _gen_inh_spikes(self, n_cells, mean_fr, std_fr, rhythmic_dict, key, fname):
                """Generates a spike raster with each train having the noise trace from
                averaging excitation. Distributes firing rates normally.
                Parameters
                ----------
                n_cells : int
                    number of spike trains
                mean_fr : float
                    mean firing rate
                std_fr : float
                    standard deviation of the firing rate
                rhythmic_dict : dict
                    dictionary with keys f - frequency, mod - depth of modulation
                key : str
                    name of the second group in the h5 file
                fname : str
                    name of file to save the raster to
                """                
                # node_ids = []
                # timestamps = []
                a, b = (0 - mean_fr) / std_fr, (100 - mean_fr) / std_fr
                d = partial(st.truncnorm.rvs, a=a, b=b, loc=mean_fr, scale=std_fr)
                
                if rhythmic_dict['f'] == "None":
                    f = h5py.File("exc_stim_spikes.h5", "r")
                    ts = f['spikes']["exc_stim"]['timestamps']
                    nid = f['spikes']["exc_stim"]['node_ids']

                    #Creates a noise trace based on the excitatory spike raster.
                    z = shift_exc_noise(ts, nid, self.params["time"]["stop"], time_shift=self.params["inh_shift"])
                    z = np.tile(z,(n_cells,1))
                    
                    writer = SonataWriter(fname, ["spikes", key], ["timestamps", "node_ids"], [np.float, np.int])
                    make_save_spikes(writer, False, d(size=n_cells), numUnits=n_cells,rateProf=z)

                else:
                    # make an array of modulated sin waves
                    # make_save_spikes should be written so that the firing rates are generated
                    #    outside instead of inside the function.
                    frs = d(size=n_cells)
                    
                    t = np.arange(0,self.params["time"]["stop"],0.001)
                    z = np.zeros((n_cells,t.shape[0]))
                    P = 0
                    for i in np.arange(0,n_cells):
                        offset = frs[i]
                        A = offset/((1/rhythmic_dict['mod'])-1)
                        z[i,:] = A*np.sin((2 * np.pi * rhythmic_dict['f'] * t)+P) + offset

                    writer = SonataWriter(fname, ["spikes", key], ["timestamps", "node_ids"], [np.float, np.int])
                    make_save_spikes(writer, False, np.ones((n_cells,1)), numUnits=n_cells,rateProf=z)

L5NetParams.json
https://github.com/latimerb/L5NeuronSimulation/blob/dea91c052102bb4c2a58b860ebed88b3af4a24bc/L5NeuronSimulation/FullSimulation/L5NetParams.json

In [ ]:
{
    "cell":
    {
        "prefix": "L5",
        "dynamic_params": null,
        "model_template": "hoc:L5PCtemplate",
        "morphology": null,
        "model_processing": null,
        "segments_file": "L5Segments.csv"
    },

    "lengths":
    {
        "basal_dist": 4649,
        "basal_prox": 483,
        "apic": 7440
    },

    "syn_density":
    {
        "exc": 2.16,
        "inh": 0.22
    },

    "n_soma_syns": 150,

    "divergence":
    {
        "exc": {"min":2, "max":8},
        "peri_inh": {"m":2.8, "s":1.9, "min":1, "max":5},
        "basal_inh": {"m":2.7, "s":1.6, "min":1 , "max":5},
        "apic_inh": {"m":12, "s":3, "min":6 , "max":18}
    },

    "groups":
    {
        "cells_per_group": 100,
        "cluster_radius": 10,
        "group_radius": 100
    },

    "file_current_clamp": {"input_file":"None"},

    "record_cellvars":
    {
        "vars": ["v"],
        "locs": ["all"]
    },

    "inh_frs":
    {
        "proximal": {"m":16.9, "s":14.3, "rhythmicity":{"f":"None", "mod":"None"}},
        "distal": {"m":3.9, "s":4.9, "rhythmicity":{"f":16, "mod":0.5}}
    },

    "time": {"start":0,"stop":100},
    "dL": 5,
    "dt": 0.1,

    "inh_shift": 2
}

raster_maker.py
https://github.com/latimerb/L5NeuronSimulation/blob/master/L5NeuronSimulation/FullSimulation/raster_maker.py#L199

In [ ]:
class SonataWriter:
    """Class used to dynamically writing spike rasters to an h5 file.
    Attributes
    ----------
    file : h5py.File
        file object being worked on
    group : h5py.Group
        gropu where the datasets reside
    datasets : dict
        datasets that are saved to the file
    Methods
    -------
    append_ds(vals, ds)
        appends the given values to the end of the given dataset
    append_repeat(ds, val, N)
        appends the given value N times to the end of the given dataset
    close()
        close the h5py file
    """
    def __init__(self, f_name, groups, datasets, types):
        """
        Parameters
        ----------
        f_name : str
            name of file location
        groups : list
            list of group names (str) that are layered into the h5py file
            in the order given.
        datasets : list
            list of dataset names (str)
        types : list
            list of data types that corresponds to the datasets list
        """        
        self.file = h5py.File(f_name, 'w')

        self.group = self.file
        for group in groups:
            self.group = self.group.create_group(group)

        self.datasets = {}
        for i, ds in enumerate(datasets):
            self.datasets[ds] = self.group.create_dataset(ds, data=[], dtype=types[i], chunks=True, maxshape=(None,))

    def append_ds(self, vals, ds):
        """appends the given values to the end of the given dataset
        Parameters
        ----------
        vals : list
            list of values to be appended to the dataset
        ds : str
            key of the dataset to append to
        """        
        length = len(self.datasets[ds])
        self.datasets[ds].resize((length + len(vals), ))
        self.datasets[ds][length:] = vals

    def append_repeat(self, ds, val, N):
        """appends the given value N times to the end of the given dataset
        Parameters
        ----------
        ds : str
            key of the dataset to append to
        val : [type]
            value to be appended N times
        N : int
            number of vals to append to the dataset
        """        
        self.append_ds([val for i in range(N)], ds)

    def close(self):
        """Closes the h5py File
        """        
        self.file.close()

def zscore(x):
    """z scores the given array"""
    return (x-np.mean(x))/np.std(x)

def minmax(x):
    """min max normalizes the given array"""
    return (x - np.min(x))/(np.max(x)-np.min(x))

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

def make_noise(num_traces=100,num_samples=4999):
    """Creates a noise trace used in generating spike rasters.
    Parameters
    ----------
    num_traces : int, optional
        number of noise traces to create (first dimension), by default 100
    num_samples : int, optional
        length of the trace (second dimension), by default 4999
    Returns
    -------
    np.array
        noise trace
    """    
    B = [0.049922035, -0.095993537, 0.050612699, -0.004408786]
    A = [1, -2.494956002,   2.017265875,  -0.522189400]
    invfn = np.zeros((num_traces,num_samples))
    for i in np.arange(0,num_traces):
        wn = np.random.normal(loc=1,
 			   scale=0.5,size=num_samples+2000)
        invfn[i,:] = minmax(ss.lfilter(B, A, wn)[2000:])+0.5                             # Create '1/f' Noise
    return invfn


def shift_exc_noise(ts, nid, seconds, time_shift=4):
    """Creates a shifted, min-max normalized average traces of the given spike raster.
    Parameters
    ----------
    ts : list
        times (float) where spikes occur
    nid : int
        node id associated with each spike
    seconds : float
        length of the raster in seconds
    time_shift : int, optional
        how many ms to shift the average trace by, by default 4
    Returns
    -------
    [type]
        [description]
    """    
    h = np.histogram(ts,bins=np.arange(0,seconds*1000,1))

    fr_prof = h[0]/(0.001*(np.max(nid)+1))
    wrap = fr_prof[-4:]
    fr_prof[4:] = fr_prof[0:-4]
    fr_prof[0:4] = wrap

    fr_prof = minmax(fr_prof)+0.5
    return fr_prof

def make_save_spikes(writer, exp, dist, numUnits=100,rateProf=None,start_id=0,start_time=0):
    """Creates and saves spikes for the given nodes using
    the provided noise trace and a random mean firing rate generated with
    the given distribution.
    Parameters
    ----------
    writer : SonataWriter
        how the spikes are saved
    exp : bool
        whether the value from dist should be fed to np.exp()
    dist : np.array()
        array of firing rates of shape (numUnits,)
    numUnits : int, optional
        number of nodes to generate spikes for, by default 100
    rateProf : np.array(), optional
        noise trace for each unit must have numUnits rows, by default None
    start_id : int, optional
        node_id that the first unit/node should be associated with, by default 0
    start_time : int, optional
        at what time the spikes should start being generated, by default 0
    """    
    
    for i in np.arange(0,numUnits):
       
        try: 
            r = rateProf[i,:]
        except:
            import pdb; pdb.set_trace()
        r[r<0] = 0#Can't have negative firing rates.
        rate_temp=[];simSpks_temp=[]

        #Multiplies the noise trace by the randomly generated firing rate.
        if exp:
            rate_temp = r*np.exp(dist[i])
        else:
            rate_temp = r*dist[i]

        numbPoints = scipy.stats.poisson(rate_temp/1000).rvs()#Poisson number of points

        simSpks=np.where(numbPoints>0)[0]

        writer.append_repeat("node_ids", i + start_id, len(simSpks))
        writer.append_ds(simSpks + start_time, "timestamps")